# Upload stimuli to cogtoolslab.org mongoDB database
Instructions:
1. open ssh tunnel to mongoDB (see instructions below)
2. change iteration name below to match study you're going to run
3. if using same stimuli from previous study, no need to change further
    1. if changing stimuli, load those in (Get Stimuli heading)
4. run all cells before "upload stimuli images" if using same stimuli. Otherwise, run whole notebook

## Set up notebook

In [1]:
# Import libraries and utils
import os
import ast
import sys
import json
import uuid
import random
import numpy as np
import pandas as pd
from operator import gt, lt

import socket
import pymongo as pm
import paramiko

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [3]:
# set up directory & file hierarchy
proj_dir = os.path.abspath('./')
print(proj_dir)

/Users/matthewcaren/research/image-scoring


In [3]:
# globals
dbname = 'tangram_construction_input'
colname = 'tanSynthesisRetrieval'
# TODO: don't require stimuli to have this!
iteration_name = 'tangramConstruction_pilot_wm_inventory'
stimdir = proj_dir + '/stimuli/double/selected'
setAdir = stimdir + '/setA'
setBdir = stimdir + '/setB'

# in experiment client code, this is location of stim images
# i.e. this dir inside tangram_construction/experiment/pilot/...
client_img_dir = 'assets'

## Get Stimuli from local directory

In [4]:
# setA
setAfiles = os.listdir(setAdir)
setAconfigs = [ f for f in setAfiles if f.endswith('.json') ]
setAstims = [ f for f in setAfiles if f.endswith('.png') ]
setAsilhouettes = [ f for f in setAstims if f.endswith('silhouette.png') ]
print('num set A', len(setAconfigs))

# set B
setBfiles = os.listdir(setBdir)
setBconfigs = [ f for f in setBfiles if f.endswith('.json') ]
setBstims = [ f for f in setBfiles if f.endswith('.png') ]
setBsilhouettes = [ f for f in setBstims if f.endswith('silhouette.png') ]
print('num set B', len(setBconfigs))

num set A 12
num set B 12


## Insert each session as a record into mongoDB

run this in your terminal...
`ssh -fNL 27017:127.0.0.1:27017 user@cogtoolslab.org`

`ssh -fNL 27017:127.0.0.1:27017 seanpaul@cogtoolslab.org`

In [5]:
# set vars
# this auth.txt file contains the password for the sketchloop user. Place it in the toplevel of the repo
auth = pd.read_csv(os.path.join(proj_dir, 'auth.txt'), header=None)
pswd = auth.values[1][1]
user = 'sketchloop'
host = 'cogtoolslab.org'  # cogtoolslab ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1:27017')

db = conn[dbname]
coll = db[colname]

In [6]:
# upload set A to mongo
trialconfiglist = []
for setchar, setconfigs in zip(['A', 'B'], [setAconfigs, setBconfigs]):
    for configfile in setconfigs:
        with open(stimdir + f'/set{setchar}/' + configfile, 'r') as f:
            config = json.load(f)

        stimfilename = os.path.basename(config['stimImgPath'])
        silhouettefilename = os.path.basename(config['stimSilhouetteImgPath'])
        # todo: do this in stimuli/generate.py, not here
        config['stimImgPath'] = client_img_dir + f'/set{setchar}/' + stimfilename
        config['stimSilhouetteImgPath'] = client_img_dir + f'/set{setchar}/' + silhouettefilename
        # separately, add the appropriate images to experiment/pilot/[client_img_dir]/set[A/B]/
        config['set'] = setchar
        #config['iteration'] = iteration_name
        trialconfiglist.append(config)

print('trialconfiglist')
print(trialconfiglist)

# add to mongo
# 'iteration' needs to match what store.js looks for...
coll.insert_one({'trials': trialconfiglist, 'iteration': iteration_name})
clear_output(wait=True)

print(f'Done inserting records into mongo! The collection name is: {coll}')
print(f'Number of entries in database: {coll.estimated_document_count()}')
print(f'example entry: \n{str(coll.find_one())[:1000]}')

Done inserting records into mongo! The collection name is: Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'tangram_construction_input'), 'tanSynthesisRetrieval')
Number of entries in database: 12
example entry: 
{'_id': ObjectId('65dedcb51c96bb3cfc34e584'), 'trials': [{'solutionTans': [{'name': 'med_triangle', 'verticesAtOrigin': [[0.0, 0.0], [1.0, 0.0], [2.0, 0.0], [1.0, 1.0]]}, {'name': 'square', 'verticesAtOrigin': [[2.0, 0.0], [1.0, 0.0], [1.0, -1.0], [2.0, -1.0]]}, {'name': 'small_triangle', 'verticesAtOrigin': [[0.0, -1.0], [1.0, 0.0], [0.0, 0.0]]}], 'solutionNumPrimitiveTans': 3, 'solutionNumTans': 2, 'tans': [{'name': 'small_triangle2', 'verticesAtOrigin': [[0.0, 0.0], [1.41421, 0.0], [0.70711, 0.70711]]}, {'name': 'parallelogram', 'verticesAtOrigin': [[0.0, 0.0], [1.41421, 0.0], [2.12132, 0.70711], [0.70711, 0.70711]]}, {'name': 'large_triangle', 'verticesAtOrigin': [[0.0, 0.0], [1.41421, 0.0], [2.82843, 0.0], [2.1

## Upload stimuli image files to server assets dir

# CAUTION: this will overwrite existing files in the server directory

In [7]:
# written by copilot and chatgpt 3.5
import paramiko
print('paramiko version', paramiko.__version__)

# server credentials
username = auth.values[2][0]
password = auth.values[2][1]
print('connecting to', host, 'with username and pw listed in ./auth.txt')

try:
    # create an SSH client instance
    ssh = paramiko.SSHClient()

    # load system host keys
    ssh.load_system_host_keys()

    # automatically add host keys
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # connect to the server
    ssh.connect(host, username=username, password=password)

    # create an sftp client from the ssh connection
    sftp = ssh.open_sftp()

    # do once for each stim set
    for setchar, setdir in zip(['A', 'B'], [setAdir, setBdir]):
        # local and remote paths
        local_path = setdir
        # TODO: create this directory if it doesn't exist
        # FIXME: a safe way to not prevent overwriting remote files??
        #        how to store stimuli from prev experiments?
        remote_path = f"/tangram_construction/experiments/pilot/assets/set{setchar}"


        # iterate over files in local directory
        print(os.getcwd())
        for file_name in os.listdir(local_path):
            if file_name.endswith('.png'):  # check if the file is a .png file
                #local_file = os.path.join(local_path, file_name)
                local_file = os.path.join(local_path, file_name)
                # for some reason, sftp needs a dot in front of the path
                remote_file = '.' + os.path.join(remote_path, file_name)
                print('uploading', local_file, 'to', remote_file)

                # upload the file
                sftp.put(local_file, remote_file)

    # close the sftp client and SSH connection
    sftp.close()
    ssh.close()

except paramiko.AuthenticationException:
    print("Authentication failed. Please check your credentials.")
except Exception as e:
    print(f"An error occurred: {e}")

paramiko version 2.8.1
connecting to cogtoolslab.org with username and pw listed in ./auth.txt
/Users/sean/Documents/code/tangram_construction/stimuli
uploading /Users/sean/Documents/code/tangram_construction/stimuli/double/selected/setA/double_parallelogram_36_stim.png to ./tangram_construction/experiments/pilot/assets/setA/double_parallelogram_36_stim.png
uploading /Users/sean/Documents/code/tangram_construction/stimuli/double/selected/setA/double_square_4_stim.png to ./tangram_construction/experiments/pilot/assets/setA/double_square_4_stim.png
uploading /Users/sean/Documents/code/tangram_construction/stimuli/double/selected/setA/double_square_4_stim_silhouette.png to ./tangram_construction/experiments/pilot/assets/setA/double_square_4_stim_silhouette.png
uploading /Users/sean/Documents/code/tangram_construction/stimuli/double/selected/setA/double_large_triangle_21_stim_silhouette.png to ./tangram_construction/experiments/pilot/assets/setA/double_large_triangle_21_stim_silhouette.png